<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_NY_mobility_poverty_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
!pip install ipython-autotime

%load_ext autotime

time: 2.22 ms (started: 2021-04-11 19:26:00 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 40.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=46623e994cee48436f76b284781446d623eb676a4643b52c7b6f3b2fa2c47828
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 138 ms (started: 2021-04-11 19:26:59 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 310 ms (started: 2021-04-11 19:26:59 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.57 s (started: 2021-04-11 19:26:59 +00:00)


In [6]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 23.2 s (started: 2021-04-11 19:27:05 +00:00)


In [7]:
# Demographic Data Chicago
!ls drive/MyDrive/big-data-project/data/clean-data/ny/ny.csv

!ls drive/MyDrive/big-data-project/data/clean-data/ny/social/2020/



drive/MyDrive/big-data-project/data/clean-data/ny/ny.csv
2020-01-01-social-distancing.csv  2020-07-02-social-distancing.csv
2020-01-02-social-distancing.csv  2020-07-03-social-distancing.csv
2020-01-03-social-distancing.csv  2020-07-04-social-distancing.csv
2020-01-04-social-distancing.csv  2020-07-05-social-distancing.csv
2020-01-05-social-distancing.csv  2020-07-06-social-distancing.csv
2020-01-06-social-distancing.csv  2020-07-07-social-distancing.csv
2020-01-07-social-distancing.csv  2020-07-08-social-distancing.csv
2020-01-08-social-distancing.csv  2020-07-09-social-distancing.csv
2020-01-09-social-distancing.csv  2020-07-10-social-distancing.csv
2020-01-10-social-distancing.csv  2020-07-11-social-distancing.csv
2020-01-11-social-distancing.csv  2020-07-12-social-distancing.csv
2020-01-12-social-distancing.csv  2020-07-13-social-distancing.csv
2020-01-13-social-distancing.csv  2020-07-14-social-distancing.csv
2020-01-14-social-distancing.csv  2020-07-15-social-distancing.csv
2020-

In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 1.65 ms (started: 2021-04-11 19:27:30 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [11]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 1.07 ms (started: 2021-04-11 19:27:49 +00:00)


In [12]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|                              8|
|360850170103|2020-02-03T00:00:...|         250|                          68|                             41|                             26|
|360050213021|2020-02-03T00:00:...|          92|                          26|                             11|                              6|
|36005

In [13]:
df_soc = spark.sql('SELECT * FROM clean_ny cl WHERE cl.device_count > 50')
df_soc.show()

+----+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
| _c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_tim

In [14]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [15]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+----+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+--------------------------+-------------------------+-------------------------+
| _c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell

In [16]:
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [17]:
#Create temp view
df_soc.createOrReplaceTempView('new_clean_ny')

time: 39.3 ms (started: 2021-04-11 19:27:51 +00:00)


In [18]:
# get mean percentage for each cbg
df_soc_home = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM new_clean_ny GROUP BY cbg')
df_soc_home.createOrReplaceTempView('mean_completely_home')
df_soc_home.show()

+------------+-------------------------------+
|         cbg|mean_completely_home_percentage|
+------------+-------------------------------+
|360050181024|              41.63320686021312|
|360810152002|              38.93963532759214|
|360470800001|             26.825793244587175|
|360810278002|              42.01491675109172|
|360810034002|              48.42883951924419|
|360470828001|             32.270041292317266|
|360850169012|               40.6464671712979|
|360050157003|             41.408718739406275|
|360810757022|             38.363939573040845|
|360810838003|              35.05670110687995|
|360050413007|              34.31001496980301|
|360050393006|              42.22696333222268|
|360810040012|             27.460860678225995|
|360850169011|              33.19255463803301|
|360470349002|              41.95617823915659|
|360811367004|             37.886860914768256|
|360610048004|             21.743794553702404|
|360470224003|              49.01492760847028|
|360050273002

TEST FOR JUST COMPLETELY HOME

In [19]:
df_mobility = df_soc_home

time: 5.09 ms (started: 2021-04-11 19:29:47 +00:00)


In [20]:
#Get mean percentage for full time work for each cbg
df_soc_ft = spark.sql('SELECT cbg, AVG(full_time_work_percentage) AS mean_full_time_work_percentage FROM new_clean_ny GROUP BY cbg')
df_soc_ft.createOrReplaceTempView('mean_fulltime_work')
df_soc_ft.show()

+------------+------------------------------+
|         cbg|mean_full_time_work_percentage|
+------------+------------------------------+
|360050181024|             4.282147446674713|
|360810152002|             6.094164646226976|
|360470800001|             6.607501564493794|
|360810278002|             3.471056136166137|
|360810034002|            3.5673737246857993|
|360470828001|            4.6941904944348325|
|360850169012|             4.541313926181154|
|360050157003|             5.653635748814089|
|360810757022|             4.901030676909368|
|360810838003|            5.3013508983699555|
|360050413007|             5.208700998794212|
|360050393006|            3.4152145071152327|
|360810040012|            4.2232846915677955|
|360850169011|             4.200477439455239|
|360470349002|             4.065173837634732|
|360811367004|            3.5819631695621545|
|360610048004|              4.01575735421718|
|360470224003|             4.527449756330783|
|360050273002|             4.87889

In [21]:
#Get mean percentage for part time work for each cbg
df_soc_pt = spark.sql('SELECT cbg, AVG(part_time_work_percentage) AS mean_part_time_work_percentage FROM new_clean_ny GROUP BY cbg')
df_soc_pt.createOrReplaceTempView('mean_parttime_work')
df_soc_pt.show()

+------------+------------------------------+
|         cbg|mean_part_time_work_percentage|
+------------+------------------------------+
|360050181024|            4.4034576781632895|
|360810152002|            6.7212455978915076|
|360470800001|             6.259708676930281|
|360810278002|             5.369135883038906|
|360810034002|             4.670583586825775|
|360470828001|            4.5416789451119985|
|360850169012|             5.491064041541418|
|360050157003|             5.682887793024334|
|360810757022|              6.14221249644029|
|360810838003|             6.161962467188746|
|360050413007|             8.750068517732844|
|360050393006|             5.006051426459162|
|360810040012|             7.016295899161092|
|360850169011|             6.626218279734299|
|360470349002|             4.909508822469144|
|360811367004|             5.230404894446388|
|360610048004|            11.944377817658237|
|360470224003|             3.705825560093583|
|360050273002|             5.87904

In [22]:
df_mobility = spark.sql('SELECT mean_completely_home.cbg, mean_completely_home.mean_completely_home_percentage, mean_fulltime_work.mean_full_time_work_percentage , mean_parttime_work.mean_part_time_work_percentage  FROM mean_completely_home INNER JOIN mean_fulltime_work ON mean_completely_home.cbg = mean_fulltime_work.cbg INNER JOIN mean_parttime_work ON mean_completely_home.cbg = mean_parttime_work.cbg')

time: 209 ms (started: 2021-04-11 19:31:13 +00:00)


In [23]:
#df_mobility.show()
df_mobility.createOrReplaceTempView('mobility')

time: 27.7 ms (started: 2021-04-11 19:31:13 +00:00)


In [24]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050001000|        0|               0.0|               0.0|
|360050001001|     7503|               0.0|12.435025989604158|
|360050002000|        0|               0.0|               0.0|
|360050002001|     2114|16.130558183538316|35.856196783349105|
|360050002002|     2168|18.911439114391143| 48.06273062730627|
|360050002003|      969| 8.152734778121776|49.742002063983485|
|360050004000|        0|               0.0|               0.0|
|360050004001|      646|32.972136222910216| 44.27244582043344|
|360050004002|     1400|25.571428571428573| 75.35714285714286|
|360050004003|     3393| 1.569905213270142| 33.65753020925435|
|360050004004|      541|15.196998123827393| 20.33271719038817|
|360050016001|     3427|  25.1531952144733|38.400933761307265|
|360050016002|      662| 62.99093655589124| 73.56495468

In [25]:
#Join mobility and poverty
result = spark.sql('SELECT demographic.*, mobility.mean_completely_home_percentage, mobility.mean_full_time_work_percentage, mobility.mean_part_time_work_percentage FROM mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
result.createOrReplaceTempView('demographic_mobility')
result.show()

+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|mean_full_time_work_percentage|mean_part_time_work_percentage|
+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|360050157003|     2002|24.925074925074924| 18.03196803196803|             41.408718739406275|             5.653635748814089|             5.682887793024334|
|360050181024|     1457|14.070006863417984| 6.314344543582704|              41.63320686021312|             4.282147446674713|            4.4034576781632895|
|360050393006|     1224|  32.6797385620915|2.5326797385620914|              42.22696333222268|            3.4152145071152327|             5.006051426459162|
|360050413007|     1188|59.570661896243294|30.723905723905

In [26]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM demographic_mobility').show()

+----------+
|count(cbg)|
+----------+
|      4927|
+----------+

time: 40.9 s (started: 2021-04-11 19:33:19 +00:00)


In [27]:
#Sort by poverty
result = spark.sql('SELECT * FROM demographic_mobility ORDER BY CAST(poverty_percentage AS float) DESC')
result.createOrReplaceTempView('demographic_mobility')
result.show()

+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|mean_full_time_work_percentage|mean_part_time_work_percentage|
+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|360610240001|     1742| 89.67653131452168|36.337543053960964|              25.17635640010764|             2.548299201025662|             4.871005673248834|
|360470910002|     1588| 80.96166341780376| 5.289672544080604|              41.16806454201998|            3.7100089552223605|             4.715055248891904|
|360050020003|     3073| 76.88787185354691|42.629352424341036|              43.17715123953364|             3.378390620523872|             4.780839402338724|
|360610143001|        5|              75.0|              4

In [28]:
# group by poverty range


time: 4.24 ms (started: 2021-04-11 19:35:52 +00:00)


In [29]:
result_df = result.toPandas()
result_df

,cbg,pop_total,poverty_percentage,perc_whiteonly,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage
0,360610240001,1742,89.67653131452168,36.337543053960964,25.176356,2.548299,4.871006
1,360470910002,1588,80.96166341780376,5.289672544080604,41.168065,3.710009,4.715055
2,360050020003,3073,76.88787185354691,42.629352424341036,43.177151,3.378391,4.780839
3,360610143001,5,75.0,40.0,41.519457,1.641875,3.461197
4,360050391006,1134,74.33862433862434,15.961199294532626,32.899632,3.321784,6.380108
...,...,...,...,...,...,...,...
4922,360610008000,0,0.0,0.0,41.349809,1.624121,2.418876
4923,360810664004,784,0.0,0.0,34.750297,6.740844,6.134852
4924,360610066001,1200,0.0,85.16666666666667,24.528302,1.886792,1.886792
4925,360610086010,0,0.0,0.0,39.947788,1.786715,3.155337


time: 1min 59s (started: 2021-04-11 19:35:52 +00:00)


In [30]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 358 ms (started: 2021-04-11 19:37:51 +00:00)


In [39]:
result_mean_df = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,100, 10))).mean().reset_index()

time: 23.6 ms (started: 2021-04-11 19:42:49 +00:00)


In [40]:
result_mean_df['poverty_percentage'] = result_mean_df['poverty_percentage'].astype(str)

time: 3.46 ms (started: 2021-04-11 19:42:52 +00:00)


In [41]:
result_mean_df['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in result_mean_df['mean_completely_home_percentage']]
result_mean_df

,poverty_percentage,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage,color
0,"(0, 10]",34.681820,5.032705,6.426703,#45a0d1
1,"(10, 20]",37.443309,4.752602,6.036222,#45a0d1
2,"(20, 30]",38.270521,4.616889,5.785856,#45a0d1
3,"(30, 40]",39.534815,4.296414,5.615374,#45a0d1
4,"(40, 50]",39.395043,4.251388,5.471793,#45a0d1
5,"(50, 60]",38.633471,4.103228,5.378113,#45a0d1
6,"(60, 70]",39.169213,3.823267,4.968284,#45a0d1
7,"(70, 80]",36.529330,3.785040,5.585335,#45a0d1
8,"(80, 90]",33.172210,3.129154,4.793030,#45a0d1


time: 25.1 ms (started: 2021-04-11 19:42:55 +00:00)


In [42]:
count = result_mean_df.shape[0]
print("{}".format(count))

9
time: 7.38 ms (started: 2021-04-11 19:43:01 +00:00)


In [43]:
viz_df = []
for col in ['mean_completely_home_percentage','mean_full_time_work_percentage','mean_part_time_work_percentage']:
        for c in range(0,count):
            viz_df.append({'poverty': c, 'column': col, 'value': result_mean_df[col][c]})
viz_df = alt.Data(values=viz_df)

time: 10.8 ms (started: 2021-04-11 19:43:05 +00:00)


In [44]:
alt.Chart(result_mean_df).mark_bar(size=30).encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 27.2 ms (started: 2021-04-11 19:43:07 +00:00)


In [45]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()



alt.Chart(...)

time: 25.1 ms (started: 2021-04-11 19:43:13 +00:00)


In [38]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')

time: 14 ms (started: 2021-04-11 19:37:52 +00:00)


In [46]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_full_time_work_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_full_time_work_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 30.2 ms (started: 2021-04-11 19:43:59 +00:00)


In [47]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_part_time_work_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_part_time_work_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 31.4 ms (started: 2021-04-11 19:44:18 +00:00)
